# Работа с базой данных для модели микроуровневого стресс-тестирования (банкротства застройщиков)

<div class="alert alert-block alert-info"><b>Блокнот предназначен для создания и обновления базы данных для модели "банкротства застройщиков"</b></div>

<div class="alert alert-block alert-warning">
Этот блокнот может вызываться из другого блокнота с параметрами - путем к файлу slite3 с базой данных для модели "банкротств застройщиков"<br>
Проверяйте, с какой именно базой работаете
</div>

***

In [34]:
%%javascript
// В этой ячейке читаем параметры вызова данного блокнота - ожидается имя рабочей базы данных 
// (отличное от имени по умолчанию). Код этой ячейки - java-script

IPython.notebook.kernel.execute("URL = '" + window.location + "'");

<IPython.core.display.Javascript object>

In [35]:
from os import path, remove
import urllib.parse as urlparse
from urllib.parse import parse_qs, urljoin
import requests
from bs4 import BeautifulSoup
import datetime as dt
import pandas as pd
import numpy as np
import re
import sqlalchemy as sa
import locale
from cmasf import pandas_sql as pds
from IPython.display import Markdown as md

# пути к рабочим файлам и каталогам
strBasePath=path.join('..', 'DB', 'bankrp.sqlite3') # путь к рабочей базе данных SQLite - может быть изменено в вызывающем блокноте
strDBPath=path.join('..', 'DB') # к каталогу с базой данных
strDB_SRC_Path=path.join('..', 'DB', 'SRC') # к каталогу с файлами данных от поставщиков 

# имена таблиц в рабочей базе данных SQLite
strInfoTable='update_info' # таблица с информацией об обновлении базы даных

strNOZA_table='noza' # название таблицы NOZA
strNOZA_data_pass='noza_columns' # таблица с названиями колонок для таблицы noza

strSPARK_table='spark' # название таблицы SPARK
strSPARK_data_pass='spark_columns' # таблица с названиями колонок для таблицы SPARK

strLOAN_table='loan' # название таблицы LOAN в базе данных SQLite
strLOAN_data_pass='loan_columns' # таблица с названиями колонок для таблицы loan

strRNKRP_table='bankrp' # таблица с фактическими банкротствами компаний

strLendTable='lending_vol' # таблица с динамикой ссудной задолженности, объемы - не используется
strLendPRC='lending' # таблица с динамикой ссудной задолженности, %
strLendReserv='lending_reserv' # таблица с резервами на потери по ссудам

In [36]:
# блок для парсинга параметров вызова данного блокнота 
# изменяет рабочую базу данных по умолчанию на указанную в параметрах вызова
parsed = urlparse.urlparse(URL)
try:
    strBasePath=parse_qs(parsed.query)['DATABASE'][0] 
except KeyError:
    pass

strMess='''<div class="alert alert-block alert-info">
<b>Рабочая база данных - {file_name}</b></div>  '''.format(file_name=strBasePath)

conWork = sa.create_engine('sqlite+pysqlite:///{db_name}'.format(db_name=strBasePath)) # connection к рабочей базе данных

# проверка наличия или создание таблицы с временем обновления данных в рабочей базе
strCreateQ='''
CREATE TABLE IF NOT EXISTS {tab_name} ("utable" TEXT, "udate" TEXT NOT NULL, PRIMARY KEY("utable"))
'''

conWork.execute(strCreateQ.format(tab_name=strInfoTable))


md(strMess)

<div class="alert alert-block alert-info">
<b>Рабочая база данных - ../DB/bankrp.sqlite3</b></div>  

## База данных NOZA

**Использование в модели:** используется на этапе подготовки данных для основной модели, а так же для расчета показателей на основе результатов основной модели  
**Источник данных:** xlsx-файл с данными, передаваемый ДОМ-РФ (NOZA)  
**Расположение:** передается в виде файла, оптимально располагается в каталоге ../DB/SRC  

**Алгоритм обработки:**
  - читаем таблицу из файла Ексель (`strNOZA_XLSX`, преполагается, что там один лист с нужной таблицей - первый)
  - выбираем нужные поля, переименовываем их в удобный формат
  - из поля `Запланированный срок ввода в эксплуатацию (текущий)` (`Input_date`) выбираем год (`year`) (предварительно поле преобразуется в тип `datetime`), создаем новое поле `year` типа `int`
  - поле `ИНН застройщика` (`inn`) преобразуется в тип `string`)
  - поле `Идентификационный номер объекта` (`obj_id`) преобразуется в тип `int`)
  - поля `'Размер уставного капитала', 'Проектная площадь жилых помещений', 'Минимальная цена 1 кв м всех квартир без учета скидок', 'Минимальная цена проектной площади жилых помещений, руб.', 'Планируемая стоимость строительства', 'Новая планируемая стоимость строительства'`(`'Capital', 'Sq_living_proj', 'Min_proce_1sqm', Min_price_living_r', 'Bld_price_proj', 'Bld_price_proj_new'`)  преобразуется в тип `float`)
  - удаляются записи с пустыми полями `ИНН застройщика` (`inn`) и `Запланированный срок ввода в эксплуатацию (текущий)` (`Input_date`)
  - в качестве индексов устанавливается поле `obj_id` и `year`.
  - полученный `DataFrame` записываем в таблицу `strNOZA_table` базы данных `strBasePath`
  
 <p>Дополнительно в ту же базу записываем словарь переименования полей из исходных в используемые в дальнейшем</p>
 
### Константы и определения:

In [37]:
strNOZA_file='noza2020.xlsx' # имя файла с исходными данными
strNOZA_inn=path.join('..', 'DB', 'Temp', 'inn_{date}.csv') # пусть к файлу с уникальными ИНН из BD NOZA для запроса BD SPARK

strNOZA_XLSX=path.join(strDB_SRC_Path, strNOZA_file) # пусть к файлу с исходными даными для BD (NOZA)
strNOZA_sheet='Строящиеся дома'

# Маркер формата исходого файла (в данном блокноте возможно обрабатывать 2 разных формата)
iNOZAFileType=2

# поля базы данных NOZA, используемые для хранения и расчетов (остальные поля импортироваться в базу не будут)
if iNOZAFileType==1:
    lstNOZA_use_cols=['ИНН застройщика',
     'Идентификационный номер объекта', # ключевое поле, уникальное
     'Наименование застройщика',
     'Организационно-правовая форма',
     'Размер уставного капитала',
     'Субъект Российской Федерации (регион)',
     'Проектная площадь жилых помещений',
     'Минимальная цена 1 кв м всех квартир без учета скидок',
     'Минимальная цена проектной площади жилых помещений, руб.',
     'Запланированный срок ввода в эксплуатацию (текущий)',
     'Планируемая стоимость строительства',
     'Новая планируемая стоимость строительства']

    # список новых имен для используемых полей базы NOZA (порядок соотв. предыдущему списку)
    lst_NOZA_names=['inn', 'obj_id', 'Name', 'OPF', 'Capital', 'REGION', 'Sq_living_proj', 'Min_proce_1sqm', 
               'Min_price_living_r', 'Input_date', 'Bld_price_proj', 'Bld_price_proj_new']
    strNOZAKey='obj_id'
else:
    
    lstNOZA_use_cols= ['Отчетная дата',
     'Идентификатор дома',
     'ИНН юрлица',
     'Наименование юрлица',
     'Регион',
     'Метод обеспечения ответственности',
     'Стоимость строительства,\nруб.',
     'Жилая площадь,\nкв. м',
     'Планируемая дата ввода']
    lst_NOZA_names=['pub_date', 'obj_id', 'inn', 'Name', 'REGION', 'Resp', 'bld_price', 'sq_living', 'Input_date']
    strNOZAKey=['pub_date', 'obj_id']
    
def NOZA_transform(pdf, file_type=iNOZAFileType):
    """функция очистки и преобразования таблицы XLS с данными NOZA в Dataframe"""

    pdfN=pdf.rename(columns=dict(zip(lstNOZA_use_cols, lst_NOZA_names))) # переименование колонок
    
    pdfN=pdfN[pdfN['inn'].notnull() & pdfN['Input_date'].notnull()] # убираем пустые ИНН и даты
    pdfN['completion_year']=pdfN['Input_date'].dt.year # выделяем из даты год запланированного ввода в эксплуатацию
    pdfN['completion_year']=pdfN['completion_year'].astype(int) 
    pdfN[strNOZAKey]=pdfN[strNOZAKey].astype(int) 
     
    if file_type==1:
        pdfN[['Capital', 'Sq_living_proj', 'Min_proce_1sqm', 
               'Min_price_living_r', 'Bld_price_proj', 'Bld_price_proj_new']]=pdfN[['Capital', 'Sq_living_proj', 'Min_proce_1sqm', 
               'Min_price_living_r', 'Bld_price_proj', 'Bld_price_proj_new']].apply(pd.to_numeric)
    else:
        pdfN[['bld_price', 'sq_living']]=pdfN[['bld_price', 'sq_living']].apply(pd.to_numeric)
        pdfN['pub_date']=pd.to_datetime(pdfN['pub_date'], format='%Y-%m-%d')
        pdfN=pdfN.loc[pdfN['pub_date']==pdfN.sort_values(by='pub_date', ascending=False)['pub_date'].iloc[0], :] # выбираем данные по последней отчетной дате
#         pdfN['pub_date']=pd.to_datetime(pdfN['pub_date'], format='%Y-%m-%d') 
    return pdfN.set_index(strNOZAKey)

### Чтение данных из файла xlsx

In [38]:
# тип ИНН - string устанавливаем именно здесь, чтобы не возиться с лидирующими нулями
if iNOZAFileType==1:
    pdfNOZA=pd.read_excel(strNOZA_XLSX, header=3, usecols=lstNOZA_use_cols,  dtype={'ИНН застройщика':str}) # читаем данные из Excel
else:
    pdfNOZA=pd.read_excel(strNOZA_XLSX, sheet_name=strNOZA_sheet, usecols=lstNOZA_use_cols,  dtype={'ИНН юрлица':str}) # читаем данные из Excel
    
print('Прочитано из файла Excel - ', pdfNOZA.shape)
# print(pdfNOZA)
pdfNOZA=NOZA_transform(pdfNOZA)
print('После очистки - ', pdfNOZA.shape)
print('='*50)
print(pdfNOZA.head())

Прочитано из файла Excel -  (111706, 9)
После очистки -  (9614, 8)
                          inn                             Name  \
pub_date   obj_id                                                
2020-03-26 2230    5321078745                      ПРОЕКТСТРОЙ   
           2569    4632005459  МКП УКС АДМИНИСТРАЦИИ Г. КУРСКА   
           4796    2311231205                 НЕМЕЦКАЯ ДЕРЕВНЯ   
           4892    2311231205                 НЕМЕЦКАЯ ДЕРЕВНЯ   
           29059   7448046410                      СЗ 10-Й ДОМ   

                                 REGION  \
pub_date   obj_id                         
2020-03-26 2230    Новгородская область   
           2569         Курская область   
           4796      Краснодарский край   
           4892      Краснодарский край   
           29059    Челябинская область   

                                                                Resp  \
pub_date   obj_id                                                      
2020-03-26 2230    Соотв

### Запись подготовленного фрейма в рабочую базу
*Используется доработанный класс `DataFrame` - `DataFrameDATA` для возможности полноценного использования `UPSERT` - добавления отсутсвующих записей и обновления существующих* 

In [ ]:
_pdf=pds.DataFrameDATA(pdfNOZA)

_pdf.to_sql(strNOZA_table, con=conWork, if_exists='upsert', chunksize=int(1e4))

print('Done writing to SQLite3 {} to table {}'.format(strBasePath, strNOZA_table), _pdf.shape)
_ser=pd.Series(dict(zip(lst_NOZA_names, lstNOZA_use_cols)), name='col_name')
_ser.at['completion_year']= 'Год планируемого ввода'
_ser.to_sql(strNOZA_data_pass, con=conWork, if_exists='replace')

print('Done writing NOZA columns names to SQLite3 {} to table {}'.format(strBasePath, strNOZA_data_pass))

_=conWork.execute('''INSERT OR REPLACE INTO {tab_info}(utable, udate) VALUES
('{table_name}', '{update_date}') '''.format(tab_info=strInfoTable, 
                                          update_date=dt.datetime.now().strftime('%Y-%m-%d'), table_name=strNOZA_table))

### Подготовка запросов для таблицы NOZA
Так как имеется два формата данных NOZA (а, возможно, будет и больше) для каждого формата создается отдельный агрегирующий запрос.  
Запрос, построенных для другого типа исходного формата данных, в базе работать не будет

In [ ]:
if iNOZAFileType==1:
    pass # пока не реализовано, основной формат = 2
elif iNOZAFileType==2:
    strNOZA_CreateAGGview='''
create view if not exists noza_ag as
select {noza_table}.pub_date, {noza_table}.inn, {noza_table}.completion_year, 
sum({noza_table}.bld_price) as price, sum({noza_table}.sq_living) as square
from {noza_table}
group by {noza_table}.pub_date, {noza_table}.inn, {noza_table}.completion_year;
    '''.format(noza_table=strNOZA_table)
else:
    pass
conWork.execute(strNOZA_CreateAGGview)

print('done create')

### Подготовка данных для запроса в базу данных SPARK

Из базы данных SPARK запрашиваем данные только для тех компаний, которые имеются в базе данных NOZA. Для запроса используем список ИНН компаний из NOZA (только уникальные).

Список сохраняем в csv (txt) файле, этот файл передаем на вход сервиса SPARK

In [39]:
strFileINN=strNOZA_inn.format(date=dt.datetime.now().strftime('%Y_%m_%d'))
# pdfNOZA.inn.unique().to_csv(strFileINN, sep=';', encoding='cp1251')
with open(strFileINN, 'w') as inn_file:
    for i in pdfNOZA.inn.unique():
        inn_file.write(i + '\n')

print('{cnt} ИНН записано в файл {name}'.format(cnt=len(pdfNOZA.inn.unique()), name= strFileINN))

3339 ИНН записано в файл ../DB/Temp/inn_2020_12_03.csv


## База данных SPARK

**Использование в модели:** для промежуточного моделирования (ROA, Z_A), для конечной модели  
**Источник данных:** xlsx-файл с данными, скачиваемый из системы СПАРК (система платная)  
**Расположение:** передается в виде файла, оптимально располагается в каталоге ../DB/SRC  

**Алгоритм обработки:**
- читаем даные из файла xlsx, там один лист с данными 
- преобразуем данные в нужный формат (см. **Особенности** нижд) 
- выбираем нужные поля (_'Дебиторская задолженность, RUB', 'Активы  всего, RUB', 'Уставный капитал , RUB', 'Кредиторская задолженность, RUB', 'Чистая прибыль (убыток), RUB', 'Рентабельность активов (ROA), %'_)
- После преобразования получившиеся колонки переименовываются в удобные для работы названия
- Корректируются типы колонок (эти 6 полей должны иметь тип `float`), поле `ИНН` (`inn`) - `string`, поле `year` - `int`
- Удяляются записи с пустыми значениями в полях `inn` и `year`
- Для получившегося `DataFrame` устанавливаются в качестве индексов комбинация полей `inn` и `year`
- Получившийся `DataFrame` записываем в таблицу `strSPARK_table` базы данных `strBasePath`


**Особенности**:
<p>База SPARK представляет собой файл MS Excel, сформированный программой выдачи интернет-ресурса</p>
<p>Особенность формата: годовые данные выводятся в колонках, при этом имя колонки содержит номер года и название поля, то есть : </p>

| 2015, Активы  всего, RUB | 2016, Активы  всего, RUB |	2017, Активы  всего, RUB | ... |  
|--------------------------|--------------------------|--------------------------|-----|
| 13 222 000 | 103 491 000 |	126 160 000 | ... |

<p>Необходимо разделить названия и года, преобразовать из широкого представления данных в длинный, то есть к виду: <p>

| year | Активы  всего, RUB | ... |  
|------|----------------|-----|
| 2015 | 13 222 000     | ... |
|------|----------------|-----|
| 2016 | 103 491 000    | ... |
|------|----------------|-----|
| 2017 | 126 160 000    | ... |
    
### Константы и определения:

In [40]:
strSPARK_XLSX=path.join(strDB_SRC_Path, 'SPARK.xlsx') # пусть к файлу с исходными даными для BD (SPARK)

# поля базы данных SPARK, используемые для хранения и расчетов 
# (кроме этих полей выбираются так же все поля, имеющие 4 цифры года в заголовке)
lstSPARK_use_cols=['Наименование',
 'Дата ликвидации',
 'Код налогоплательщика',
 'Регион регистрации']

# список новых имен для используемых полей базы SPARK (порядок соотв. предыдущему списку)
lst_SPARK_names=['Name', 'Cancel_date', 'inn', 'REGION']

# словарь переименования полей базы данных SPARK, содержащих год в заголовке (цифры года уйдут в новое поле индекса)
dctDFS={'receivables':'Дебиторская задолженность, RUB', 'actives':'Активы  всего, RUB', 
            'capital':'Уставный капитал , RUB', 'acc_pay':'Кредиторская задолженность, RUB', 
            'profit':r'Чистая прибыль \(убыток\), RUB', 'roa':r'Рентабельность активов \(ROA\), %'}

lstSPARKIndexFields=['year', 'inn'] # список ключевых полей для SPARK

def SPARK_transform(pdf):
    """ подготовка базы данных СПАРК для использования в обучающей последовательности """
    
    dctRes=dict()
    
    # p_name=pdfSPARK[['INN', 'Name', 'REGION']].drop_duplicates().set_index('INN')

    _use_cols1=lstSPARK_use_cols+[c for c in pdf.columns if re.match(r'\d{4}', c)] # список: заданные колонки и колонки, начинающиеся с года
    pdfSrc=pdf[_use_cols1].rename(columns=dict(zip(lstSPARK_use_cols, lst_SPARK_names))) # переименовываем в новые названия для базы 

    for k, v in dctDFS.items():
        _use_cols=['inn',]+[c for c in pdf.columns if re.search(v, c)] # список: ИНН и колонки, начинающиеся с года
        _pdf=pdfSrc[pdfSrc['inn'].notnull()] # новый ДатаФрейм, очищенный от пропусков ИНН
        _pdf=_pdf[_use_cols].set_index('inn') # ДатаФрейм с ИНН и одним показателем по нескольким годам
        _pdf.columns=_pdf.columns.str[:4].astype(int)
        _pdf=_pdf.unstack()
        _pdf.index.levels[0].astype(int)
        _pdf.name=k
        dctRes.setdefault(k, _pdf)
        
    pdfRes=pd.DataFrame(dctRes).join(pdfSrc[lst_SPARK_names].set_index('inn'), on='inn')
    pdfRes.index.names=lstSPARKIndexFields
    return pdfRes

### Чтение данных из файла

In [41]:
# для заполнения СПАРК ист-данными - раскомментировать следующее
# strSPARK_hist1='СПАРК_Выборка_компаний_20200723_1612.xlsx'
# strSPARK_hist2='СПАРК_Выборка_компаний_20200723_1609.xlsx'
# strSPARK_hist3='СПАРК_Выборка_компаний_20200723_1614.xlsx' # !!! глючит!! дубликаты в базе !!!

# strSP_path=path.join('DB', 'SRC', strSPARK_hist2) # для заполнения базы СПАРК данными предшествующих годов (база позволяет выкачивать по 5 лет за один раз)
# pdfSPARK=pd.read_excel(strSP_path, header=3, dtype={'Код налогоплательщика':str})
#----------------------------------------

# для заполнения последней точки - раскомментировать следующее
pdfSPARK=pd.read_excel(strSPARK_XLSX, header=3, dtype={'Код налогоплательщика':str})
#---------------------------------------

print('Прочитано из файла Excel - ', pdfSPARK.shape)
print(pdfSPARK.shape)
# pdfSPARK[pdfSPARK['Код налогоплательщика'].duplicated(keep=False)].to_csv('dupls.csv', sep=';')
pdfSPARK=SPARK_transform(pdfSPARK)
print('После очистки - ', pdfSPARK.shape)
print('='*50)
print(pdfSPARK.head())

Прочитано из файла Excel -  (8674, 39)
(8674, 39)
После очистки -  (43440, 9)
                 receivables      actives    capital      acc_pay     profit  \
year inn                                                                       
2015 3906245291          NaN          NaN        NaN          NaN        NaN   
     3801132195    3414000.0   13222000.0        NaN    1656000.0   516000.0   
     5050102110  139989000.0  973110000.0    10000.0  512847000.0  1556000.0   
     7448046410          NaN      10000.0        NaN          NaN        NaN   
     1840032987   17597000.0   62803000.0  5050000.0   46430000.0    34000.0   

                    roa                                            Name  \
year inn                                                                  
2015 3906245291     NaN                     1 СТРОИТЕЛЬНЫЙ ХОЛДИНГ, ООО   
     3801132195  0.0781        1-ДСК, ООО СПЕЦИАЛИЗИРОВАННЫЙ ЗАСТРОЙЩИК   
     5050102110  0.0020                                 10 КВ

### Запись подготовленного фрейма в рабочую базу

*Используется доработанный класс `DataFrame` - `DataFrameDATA` для возможности полноценного использования `UPSERT` - добавления отсутсвующих записей и обновления существующих* 

In [ ]:
_pdf=pds.DataFrameDATA(pdfSPARK.copy())
try:
    _pdf['Cancel_date']=_pdf['Cancel_date'].dt.strftime('%Y-%m-%d') # преобразование в строку чтобы убрать NaT (в SQLite все равно в строке лежит)
except AttributeError: # уже в строке
    pass
    
_pdf.to_sql(strSPARK_table, con=conWork, if_exists='upsert', chunksize=int(1e4))

print('Done writing to SQLite3 {} to table {}'.format(strBasePath, strSPARK_table), _pdf.shape)

dctSPARK_pass=dict(zip(lst_SPARK_names, lstSPARK_use_cols))
dctSPARK_pass.update({k:v.replace('\\', '') for k, v in dctDFS.items()})
pd.Series(dctSPARK_pass, name='col_name').to_sql(strSPARK_data_pass, con=conWork, if_exists='replace')

print('Done writing SPARK columns names to SQLite3 {} to table {}'.format(strBasePath, strSPARK_data_pass))

_=conWork.execute('''INSERT OR REPLACE INTO {tab_info}(utable, udate) VALUES
('{table_name}', '{update_date}') '''.format(tab_info=strInfoTable, 
                                          update_date=dt.datetime.now().strftime('%Y-%m-%d'), table_name=strSPARK_table))

### Подготовка запросов для таблицы SPARK
Данные модель будет получать посредством этого запроса

Готовим два варианта запроса:
  1. ROA компании используется из базы данных, не расчитывается; Z_A компании считается по формуле:
  
$Z\_A = |\frac{(КРЕДИТОРСКАЯ\_ЗАДОЛЖЕННОСТЬ - ДЕБИТОРСКАЯ\_ЗАДОЛЖЕННОСТЬ)}{АКТИВЫ\_ВСЕГО}|$
      
      
      
  2. ROA компании расчитывается по формуле: 
  
$ROA = ЧИСТАЯ\_ПРИБЫЛЬ - АКТИВЫ\_ВСЕГО$
      
      Z_A компании считается по формуле:
      
$Z\_A=|\dfrac{(КРЕДИТОРСКАЯ\_ЗАДОЛЖЕННОСТЬ - ДЕБИТОРСКАЯ\_ЗАДОЛЖЕННОСТЬ)}{АКТИВЫ\_ВСЕГО}|$

In [ ]:
strSPARK_CreateAGGview='''
create view if not exists spark_ag as
select {spark_table}.inn, {spark_table}.year, {spark_table}.capital, {spark_table}.Cancel_date, 
{spark_table}.roa as ROA, 
abs( ({spark_table}.acc_pay-{spark_table}.receivables )/{spark_table}.actives) as Z_A
from {spark_table};
    '''.format(spark_table=strSPARK_table)

strSPARK_CreateAGGview_ROA_CALC='''
create view if not exists spark_ag_roa_calc as
select {spark_table}.inn, {spark_table}.year, {spark_table}.capital, {spark_table}.Cancel_date, 
({spark_table}.profit - {spark_table}.actives) as ROA, 
abs( ({spark_table}.acc_pay-{spark_table}.receivables )/{spark_table}.actives) as Z_A
from {spark_table};
    '''.format(spark_table=strSPARK_table)

conWork.execute(strSPARK_CreateAGGview)
conWork.execute(strSPARK_CreateAGGview_ROA_CALC)

print('done create')

## База данных кредитной ставки с сайта ЦБР

**Использование в модели:** для промежуточного моделирования (LOAN)  
**Источник данных:** http://cbr.ru/statistics/bank_sector/int_rat/   
&nbsp;&nbsp;&nbsp;&nbsp;На странице источника ищем ссылки с ключевым текстом "Сведения по кредитам в рублях, долларах США и евро нефинансовым организациям"  
**Расположение:** Интернет  

**Алгоритм обработки:**
- находим на сайте ЦБ РФ ссылки по ключевому тексту
- читаем по ссылкам файлы xlsx, нужный лист - `ставки_руб`, нужный столбец - `F`
- преобразуем поля к нужному типу, записывем в таблицу в базе данных

### Константы и определения:

In [42]:
strCBR_LOAN_SRC=r'http://cbr.ru/statistics/bank_sector/int_rat/' # основной адрес с источниками данных

strKeyText='Сведения по кредитам в рублях, долларах США и евро нефинансовым организациям'.replace(' ', '\s+') # ключевой текст для поиска нужныйх файлов

re_t=re.compile(strKeyText)

def find_link(div_tags):
    # поиск нужного файла, определенияего даты
    for d in div_tags:
        if d.find('span', class_='document-regular_name_visible', text=re_t) and d.find('div', class_='document-regular_comment', text=re.compile('в целом по Российской Федерации'.replace(' ', '\s'))):
            pub_date=dt.datetime.strptime(d.find('div', class_='document-regular_date').text.strip(), '%d.%m.%Y')
            a=d.find('a')
            
            print('Найден файл с нужными данныи, дата публикации {date}, ссылка - {link}'.format(date=pub_date.date(), link=a['href']))
            return pub_date, a['href']


def convert_date(x):
    # преобразование даты (чтобы не возиться с локалями)
    dct_month_transl={'Январь':1, 'Февраль':2, 'Март':3, 'Апрель':4, 'Май':5, 'Июнь':6, 
                  'Июль':7, 'Август':8, 'Сентябрь':9, 'Октябрь':10, 'Ноябрь':11, 'Декабрь':12}
    dtp=x.split(' ')
    return dt.date(int(dtp[1]), dct_month_transl[dtp[0]], 1)

### Чтение данных  
- читаем страницу из Сети, определяем ссылки
- читаем файл по ссылке, преобразуем в нужные форматы, получаем фрейм с данными

#### Чтение исторических данных по ставке (с 2010 по 2013)  - для выполнения сделать ячейку "кодовой" и выполнить
```Python
    
def get_historical_loan_data(lstYs=list(range(10, 14))):
    dct_month_transl={'Январь':1, 'Февраль':2, 'Март':3, 'Апрель':4, 'Май':5, 'Июнь':6, 
                  'Июль':7, 'Август':8, 'Сентябрь':9, 'Октябрь':10, 'Ноябрь':11, 'Декабрь':12}
    
    strHistLoanURL=r'http://cbr.ru/statistics/b_sector/rates_cr-no-r_{year02}/'
    
    strsrc='.*(' + ')|('.join(list(dct_month_transl.keys())) + ').*'
    lst_res=[]
    
    for y in lstYs:
        resp_l=requests.get(strHistLoanURL.format(year02=y))
        loan_sp=BeautifulSoup(resp_l.text, 'html.parser')
        loan_data=loan_sp.find('table', class_='data')
        try:
            trs=loan_data.find_all('tr')#, text=re.compile(strsrc))
        except AttributeError:
            print('for year {year} has no data'.format(year=y))
            continue
            
        for i, r in enumerate(trs):
            f_month=re.search(strsrc, r.text)
            if f_month:
                hs=r.find_all('td')
                lst_res.append({'date':dt.date(day=1, month=dct_month_transl[f_month[0]], year=2000+y), 'loan_nonfin':hs[5].text})

    return lst_res

_hist_loan=pds.DataFrameDATA(get_historical_loan_data()).set_index('date')
_hist_loan=_hist_loan.applymap(lambda x: float(x.replace(',', '.')))

_hist_loan.to_sql(strLOAN_table, con=conWork, if_exists='upsert')
print('writes to LOAN table {} records'.format(_hist_loan.shape[0]))
```

In [43]:
resp=requests.get(strCBR_LOAN_SRC)

loan_soup=BeautifulSoup(resp.text)

div_docs=loan_soup.findAll('div', class_='document-regular')

print('Найдено инфо-тегов', len(div_docs))

loan_pub_date, loan_href=find_link(div_docs)

Найдено инфо-тегов 31
Найден файл с нужными данныи, дата публикации 2020-11-12, ссылка - /vfs/statistics/pdko/int_rat/loans_nonfin.xlsx


In [44]:
pdf_loan=pd.read_excel(urljoin(strCBR_LOAN_SRC, loan_href), sheet_name='ставки_руб.', 
                       skiprows=4, usecols='A, F').dropna()

pdf_loan.columns=['date', 'loan_nonfin']
pdf_loan['date']=pdf_loan['date'].apply(convert_date)
pdf_loan.set_index('date', inplace=True)
pdf_loan

,loan_nonfin
date,
2014-01-01,9.15
2014-02-01,9.43
2014-03-01,10.29
2014-04-01,10.53
2014-05-01,10.60
...,...
2020-05-01,7.28
2020-06-01,6.89
2020-07-01,6.27


### Запись фрейма в базу данных

In [19]:
_pdf=pds.DataFrameDATA(pdf_loan.copy())
_pdf.to_sql(strLOAN_table, con=conWork, if_exists='upsert', chunksize=int(1e4))

pds.DataFrameDATA([{'index':'date', 'col_name': '''Средневзвешенные процентные ставки по кредитам, 
предоставленным кредитными организациями нефинансовым организациям в рублях, до 1 года, 
включая "до востребования"'''}] ).set_index('index').to_sql(strLOAN_data_pass, con=conWork, if_exists='upsert')

_=conWork.execute('''INSERT OR REPLACE INTO {tab_info}(utable, udate) VALUES
('{table_name}', '{update_date}') '''.format(tab_info=strInfoTable, 
                                          update_date=dt.datetime.now().strftime('%Y-%m-%d'), 
                                             table_name=strLOAN_table))

создание таблицы с описанием колонок loan и запроса выдачи данных

In [8]:
strLOAN_CreateView_LOAN='''
create view if not exists loan_agg as
select {loan_table}.date, {loan_table}.loan_nonfin
from {loan_table};
    '''.format(loan_table=strLOAN_table)

_=conWork.execute(strLOAN_CreateView_LOAN)

## Данные по ссудам (таблица 47) и резервам (таблица 51) из "Обзор банковского сектора Российской Федерации"


**Использование в модели:** для промежуточного моделирования (LOAN)  
**Источник данных:** http://cbr.ru/statistics/bank_sector/   
&nbsp;&nbsp;Читаем только Excel-файлы (ист. данные из pdf-файлов уже скачены заранее и лежат в базе). Если надо создать базу заново, то все намного сложнее. Используемые листы:
- T47, Кредитный риск, Динамика структуры ссудной задолженности банковского сектора
- T51, Характеристики резерва на возможные потери по ссудам (РВПС) по различным группам кредитного риска
**Расположение:** Интернет  

**Алгоритм обработки:**
- находим на сайте ЦБ РФ нужные ссылки  
- выбираем файл с последней датой, экспресс-обзор игнорируется  
- читаем таблицы 47 и 51, из 47-й читаем проценты, из 51-й читаем только "Фактически сформированный резерв, в % от ссудной задолженности данной категории качества"
- формируем фреймы, записываем в базу данных

### Константы и определения:

In [45]:
strURL_=r'http://cbr.ru/statistics/bank_sector/review/'

strLendTable='lending'
strLendPRC='lending_persc'
strLendReserv='lending_reserv'

# словарь для конверсии русских дат в нормальные (чтобы не возиться с локалями)
dct_month={'январь':'Jan', 'февраль':'Feb', 'март':'Mar', 'апрель':'Apr', 'май':'May', 'июнь':'Jun', 
           'июль':'Jul', 'август':'Aug', 'сентябрь':'Sep', 'октябрь':'Oct', 'ноябрь':'Nov', 'декабрь':'Dec'}


# def iterate_group(iterator, count):
#     itr = iter(iterator)
#     for i in range(0, len(iterator), count):
#         yield iterator[i:i + count]

### Чтение данных

In [46]:
resp=requests.get(strURL_)
ln2_soup=BeautifulSoup(resp.text)

print(resp)

<Response [200]>


Строим список нужный файлов с датой публикацией и текстом ссылки

In [63]:
strCBR=r'http://cbr.ru'

a_refs=ln2_soup.findAll('a', class_='versions_item')

lst_xsl_hrefs=[]

for a in a_refs:
    try:
        iyear=re.search(r'(?m)(?P<year>\d{4})', a['title']).group('year') 
        strSM=r'(?m)(?P<month>({months}))'.format(months=')|('.join(dct_month.keys()))
        str_month=re.search(strSM, a.text).group('month') 
#         print(str_month)
    except:
        print('some error - ', a['title'], a['href'], a.text)
        continue
        
    dtpub=dt.datetime.strptime('1 {month} {year}'.format(month=dct_month[str_month], year= iyear), '%d %b %Y')
    
    if re.search(r'\.xls\w', a['href']) and not bool(re.search('экспресс-выпуск', a.text)):
        lst_xsl_hrefs.append({'url':strCBR + a['href'], 'date':dtpub, 'title':a.text})
        
pdf_files=pd.DataFrame(lst_xsl_hrefs).sort_values(by='date', ascending=False)
print('='*70)
pdf_files

some error -  Опубликовано 27 ноября 2020 /Collection/Collection/File/31587/obs_218.xlsx экспресс-выпуск
some error -  Опубликовано 03 декабря 2019 /Collection/Collection/File/25496/Metodol_21.pdf Выпуск 21
some error -  Опубликовано 17 августа 2016 /Collection/Collection/File/8547/Metodol_20.pdf Выпуск 20
some error -  Опубликовано 01 апреля 2014 /Collection/Collection/File/8548/Metodol_19.pdf Выпуск 19
some error -  Опубликовано 18 марта 2013 /Collection/Collection/File/8549/Metodol_18.pdf Выпуск 18
some error -  Опубликовано 06 августа 2012 /Collection/Collection/File/8550/Metodol_17.pdf Выпуск 17


,url,date,title
0,http://cbr.ru/Collection/Collection/File/31579...,2020-11-01,2020 г.: ноябрь
1,http://cbr.ru/Collection/Collection/File/29261...,2020-10-01,2020 г.: октябрь
2,http://cbr.ru/Collection/Collection/File/29170...,2020-09-01,2020 г.: сентябрь
3,http://cbr.ru/Collection/Collection/File/28028...,2020-08-01,2020 г.: август
4,http://cbr.ru/Collection/Collection/File/27973...,2020-07-01,2020 г.: июль
5,http://cbr.ru/Collection/Collection/File/27902...,2020-06-01,2020 г.: июнь
6,http://cbr.ru/Collection/Collection/File/27838...,2020-05-01,2020 г.: май
7,http://cbr.ru/Collection/Collection/File/27388...,2020-03-01,2020 г.: март
8,http://cbr.ru/Collection/Collection/File/25668...,2020-01-01,2020 г.: январь


#### Читаем данные по выбранной ссылке (по споследней дате, можно изменить)

Читаем три таблицы в три фрейма

In [92]:
iLoc=0

print('Для обновления берем файл (один) с последней датой - ', pdf_files.iloc[iLoc, 1])

href=pdf_files.iloc[iLoc, 0]
print('Читаем по ссылке - ', href)

def read_excel_bank_review(strUrl):
    lst_head=['standart', 'nonstandart', 'doubtful', 'problem', 'hopeless']
    
    def upwork(pdf, lstHead=lst_head):
        _p=pdf.T
        _p.columns=lstHead
        _p.index.name='date'
        _p=_p.astype(float)
        return _p
        
    def read_t51(strUrl):
        _p=pd.read_excel(strUrl, sheet_name='T51', header=3,
                        usecols='A:K', nrows=4).rename(columns={'Unnamed: 0':'row_head'}).set_index('row_head')
#         _p.columns=[re.sub('^0', '', pd.to_datetime(c).strftime('%d.%m.%y'), count=1) for c in _p.columns]
        _p.columns=[pd.to_datetime(re.search('\d{,2}\.\d{,2}\.\d{,4}', c)[0], format='%d.%m.%y') for c in _p.columns]
        _p=_p.iloc[:, _p.shape[1]//2:]
        
        _p=upwork(_p, lstHead=lst_head[1:])
        return pds.DataFrameDATA(_p)
    
    def read_t47(strUrl):
        _p=pd.read_excel(strUrl, sheet_name='T47', header=5, 
                        usecols='C:M', nrows=5, 
                        skiprows=[6,] ).rename(columns={'Unnamed: 2':'row_head'}).set_index('row_head')

        lst_new_col=[c for c in _p.columns if not re.search('Unnamed\:', str(c))]
        _p1 = _p[[c for c in _p.columns if c not in lst_new_col]]
        _p  = _p[lst_new_col]
        
#         _p.columns=[re.sub('^0', '', c.strftime('%d.%m.%y'), count=1) for c in _p.columns]
        _p.columns=[pd.to_datetime(c, format='%d.%m.%y') for c in _p.columns]

        _p1.columns=_p.columns
        
        _p=upwork(_p)
        _p1=upwork(_p1)
        return pds.DataFrameDATA(_p1), pds.DataFrameDATA(_p)
    
    t47_prc, t47=read_t47(strUrl)
    t51=read_t51(strUrl)
    
    return t47_prc, t47, t51


p47_prct, _, p51=read_excel_bank_review(href)

p47_prct.index=p47_prct.index.strftime('%d.%m.%y').str.replace('^0', '', 1)
p51.index=p51.index.strftime('%d.%m.%y').str.replace('^0', '', 1)

print('47', p47_prct)

print('51', p51)

Для обновления берем файл (один) с последней датой -  2020-11-01 00:00:00
Читаем по ссылке -  http://cbr.ru/Collection/Collection/File/31579/obs_217.xlsx
47           standart  nonstandart  doubtful   problem  hopeless
date                                                         
1.01.19  43.987777    39.239509  6.649249  2.934367  7.189099
1.01.20  41.445623    42.533939  6.727371  2.478985  6.814082
1.07.20  40.588374    42.634103  7.201736  2.631722  6.944064
1.09.20  39.197872    44.246431  7.009018  2.570800  6.975878
1.10.20  39.181985    44.505869  7.010093  2.537782  6.764272
51          nonstandart  doubtful  problem  hopeless
date                                             
1.01.19         2.17     18.08    46.99     81.28
1.01.20         2.37     18.33    50.39     82.09
1.07.20         2.38     18.04    51.27     83.71
1.09.20         2.44     18.26    51.80     83.75
1.10.20         2.49     18.44    52.68     84.37


### Запись в базу данных информации по ссудам и резервам, обновление таблицы апдейтов

In [93]:
# p47.to_sql(strLendTable, con=conWork, if_exists='upsert')
# _=conWork.execute('''INSERT OR REPLACE INTO {tab_info}(utable, udate) VALUES
# ('{table_name}', '{update_date}') '''.format(tab_info=strInfoTable, 
#                                           update_date=dt.datetime.now().strftime('%Y-%m-%d'), 
#                                              table_name=strLendTable))

p47_prct.to_sql(strLendPRC, con=conWork, if_exists='upsert')
_=conWork.execute('''INSERT OR REPLACE INTO {tab_info}(utable, udate) VALUES
('{table_name}', '{update_date}') '''.format(tab_info=strInfoTable, 
                                          update_date=dt.datetime.now().strftime('%Y-%m-%d'), 
                                             table_name=strLendPRC))

p51.to_sql(strLendReserv, con=conWork, if_exists='upsert')
_=conWork.execute('''INSERT OR REPLACE INTO {tab_info}(utable, udate) VALUES
('{table_name}', '{update_date}') '''.format(tab_info=strInfoTable, 
                                          update_date=dt.datetime.now().strftime('%Y-%m-%d'), 
                                             table_name=strLendReserv))

## База данных с фактическими банкротствами компаний

**Использование в модели:** для обучение конечной модели  
**Источник данных:** https://www.kp.ru/guide/bankrotstvo-predprijatija.html, файл `Bnk_graphics.xlsx`, в лист `base_bnk`
&nbsp;&nbsp;Данные скачиваются отдельно, специальной программой (используется silenium)  
**Расположение:** файл xlsx в ../DB/SRC/    

**Алгоритм обработки:** Читаем данные из этого файла, формируем таблицу и записываем ее в базу данных.

### Константы и определения:

In [94]:
strBNKRP_path=path.join(strDB_SRC_Path, 'Bnk_graphics.xlsx') # пусть к файлу с фактическими банкротствами компаний

#  DB/SRC/Bnk_graphics.xlsx, лист `base_bnk` данные получены из  возможна перестройка на внутреннюю базу данных заказчика)

### Чтение данных

In [95]:
pdf_BankrupsFact=pd.read_excel(strBNKRP_path, sheet_name='base_bnk', usecols='G, P', 
                               header=5, converters={'inn':str,'y_pub':int})
# pdf_BankrupsFact['inn']=pdf_BankrupsFact['inn'].astype(str)
# pdf_BankrupsFact['y_pub']=pdf_BankrupsFact['y_pub'].astype(int)
pdf_BankrupsFact=pdf_BankrupsFact.set_index('inn')
pdf_BankrupsFact=pdf_BankrupsFact.loc[pdf_BankrupsFact['y_pub']>2008].sort_index()
pdf_BankrupsFact.head()

,y_pub
inn,
1001000647,2019
1001000809,2012
1001000975,2015
1001001295,2014
1001003560,2012


In [96]:
pdf_BankrupsFact=pdf_BankrupsFact.reset_index().rename(columns={'y_pub':'year'}).dropna()
pdf_BankrupsFact['Y']=1
pdf_BankrupsFact=pdf_BankrupsFact[['inn', 'year', 'Y']].set_index(['inn', 'year'])
pdf_BankrupsFact

,,Y
inn,year,
1001000647,2019,1
1001000809,2012,1
1001000975,2015,1
1001001295,2014,1
1001003560,2012,1
...,...,...
9901007664,2012,1
9901034467,2015,1
9999999991,2016,1


сохраняем полученные данные в базе данных

In [33]:
_pdf=pds.DataFrameDATA(pdf_BankrupsFact.copy())
_pdf.to_sql(strRNKRP_table, con=conWork, if_exists='upsert', chunksize=int(1e4))

_=conWork.execute('''INSERT OR REPLACE INTO {tab_info}(utable, udate) VALUES
('{table_name}', '{update_date}') '''.format(tab_info=strInfoTable, 
                                          update_date=dt.datetime.now().strftime('%Y-%m-%d'), 
                                             table_name=strRNKRP_table))
print('Done writing ', strRNKRP_table, 'table')

Done writing  bankrp table


## Окончание работы: очистка и сжатие конечного файла SQLite3

In [37]:
# ВНИМАНИЕ!!! УДАЛЕНИЕ ФАЙЛА С ИНН ДЛЯ ЗАПРОСА БАЗЫ СПАРК!!!
try:
    remove(strFileINN)
except FileNotFoundError:
    pass
#==========================================================

import sqlite3
conn = sqlite3.connect(strBasePath, isolation_level=None)
conn.execute("VACUUM") # сжатие базы данных
conn.close()

strMD='''
<div class="alert alert-block alert-success">
<b>База данных - {bd_path} успешно обновлена<br>
<a href="{bankrupt_prob}?DATABASE={bd_path}" target="_blank">Перейти в Модель микроуровневого стресс-тестирования</a></b><br>
</div>
'''.format(bankrupt_prob='bankrupt_prob.ipynb', bd_path=strBasePath)
md(strMD)


<div class="alert alert-block alert-success">
<b>База данных - ../DB/bankrp.sqlite3 успешно обновлена<br>
<a href="bankrupt_prob.ipynb?DATABASE=../DB/bankrp.sqlite3" target="_blank">Перейти в Модель микроуровневого стресс-тестирования</a></b><br>
</div>
